# Import the necessaries libraries

In [2]:
# Verify which python environment we are running the notebook on

!pyenv version
!pip install kaggle
!pip install matplotlib
!pip install spacy
!python -m spacy download en_core_web_md


wino (set by /home/jackzhong12/code/sabrinaauger/wino/.python-version)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 3.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
#packages import

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import re
#from kaggle.api.kaggle_api_extended import KaggleApi
#import kaggle
import os
from scipy import stats
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize

import spacy
os.environ['KAGGLE_USERNAME'] = 'sabrinaauger'
os.environ['KAGGLE_KEY'] = '8db79b4ccae98a2c74af940e1f1b7c35'
os.getcwd()


'/home/jackzhong12/code/sabrinaauger/wino/notebooks'

In [4]:
'''
Loading Kaggle username and key from user's own .kaggle/kaggle.json

def load_kaggle_credentials():
    kaggle_creds_path = os.path.expanduser("~/.kaggle/kaggle.json")

    if os.path.exists(kaggle_creds_path):
        with open(kaggle_creds_path) as f:
            kaggle_creds = json.load(f)
            return kaggle_creds.get('username'), kaggle_creds.get('key')

    return None, None

Load Kaggle credentials
kaggle_username, kaggle_key = load_kaggle_credentials()

Authenticate with Kaggle API
api = KaggleApi()
api.authenticate()

#REMOVE AFTER TESTING: Print Kaggle username for testing
print(f"Authenticated as: {kaggle_username}")
'''


'\nLoading Kaggle username and key from user\'s own .kaggle/kaggle.json\n\ndef load_kaggle_credentials():\n    kaggle_creds_path = os.path.expanduser("~/.kaggle/kaggle.json")\n\n    if os.path.exists(kaggle_creds_path):\n        with open(kaggle_creds_path) as f:\n            kaggle_creds = json.load(f)\n            return kaggle_creds.get(\'username\'), kaggle_creds.get(\'key\')\n\n    return None, None\n\nLoad Kaggle credentials\nkaggle_username, kaggle_key = load_kaggle_credentials()\n\nAuthenticate with Kaggle API\napi = KaggleApi()\napi.authenticate()\n\n#REMOVE AFTER TESTING: Print Kaggle username for testing\nprint(f"Authenticated as: {kaggle_username}")\n'

## Import the dataset from Kaggle

In [5]:
#Retrieve the data from Kaggle platform
!kaggle datasets download -d zynicide/wine-reviews


wine-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#Unzip the data zip file
#!unzip wine-reviews.zip


In [7]:
#Retrieve the data from csv unzipped files into two dataframes
wine1_df=pd.read_csv("~/code/sabrinaauger/wino/data/raw_data/winemag-data-130k-v2.csv")


In [8]:
wine2_df=pd.read_csv("~/code/sabrinaauger/wino/data/raw_data/winemag-data_first150k.csv")


# Data vizualisation

In [9]:
#Columns overview of both datasets
wine1_df.columns, wine2_df.columns


(Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'taster_name',
        'taster_twitter_handle', 'title', 'variety', 'winery'],
       dtype='object'),
 Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
       dtype='object'))

In [10]:
#Analysing the variety column and the unique value
wine1_df['variety'].unique()


array(['White Blend', 'Portuguese Red', 'Pinot Gris', 'Riesling',
       'Pinot Noir', 'Tempranillo-Merlot', 'Frappato', 'Gewürztraminer',
       'Cabernet Sauvignon', 'Nerello Mascalese', 'Chardonnay', 'Malbec',
       'Tempranillo Blend', 'Meritage', 'Red Blend', 'Merlot',
       "Nero d'Avola", 'Chenin Blanc', 'Gamay', 'Sauvignon Blanc',
       'Viognier-Chardonnay', 'Primitivo', 'Catarratto', 'Inzolia',
       'Petit Verdot', 'Monica', 'Bordeaux-style White Blend', 'Grillo',
       'Sangiovese', 'Cabernet Franc', 'Champagne Blend',
       'Bordeaux-style Red Blend', 'Aglianico', 'Petite Sirah',
       'Touriga Nacional', 'Carmenère', 'Albariño', 'Petit Manseng',
       'Rosé', 'Zinfandel', 'Vernaccia', 'Rosato', 'Grüner Veltliner',
       'Viognier', 'Vermentino', 'Grenache Blanc', 'Syrah', 'Nebbiolo',
       'Shiraz-Cabernet Sauvignon', 'Pinot Blanc', 'Alsace white blend',
       'Barbera', 'Rhône-style Red Blend', 'Portuguese White', 'Graciano',
       'Tannat-Cabernet', 'Sauvign

In [11]:
# Looking at the description of 1 specific wine
index_of_record_to_view = 3
des = wine2_df.loc[index_of_record_to_view, 'description']
des


"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032."

In [12]:
wine1_df.shape, wine2_df.shape


((129971, 14), (150930, 11))

# Preprocessing the data 

## Cleaning wine1_df and wine2_df dataset 

### Removing Unuseful columns for the recommendation system

In [13]:
#Remove both columns taster name and taster twitter_handle as they are not relevant for our analysis
wine_review1_df=wine1_df.drop(columns=["taster_twitter_handle","taster_name","Unnamed: 0"])
wine_review2_df=wine2_df.drop(columns=["Unnamed: 0"])
#wine_review2_df.head()


In [14]:
#Extracting the year from the column title in wine1_df

# Extract the year from title into a dedicated column using regular expression
#regular instruction (\d{4}) simply looks for four consecutive digits in the strings
wine_review1_df['Year']=wine_review1_df['title'].str.extract(r'(\d{4})')

missing_year=wine_review1_df['Year'].isna().sum()
print(f'Missing data in year variable : {missing_year} - Decision taken: not adding this additional variable')
wine_review1_df.drop(columns='Year', inplace=True)


Missing data in year variable : 4609 - Decision taken: not adding this additional variable


### Duplicates

In [15]:
#checking if there is any overall duplicates in both datasets
print(wine_review1_df.duplicated().sum())
print(wine_review2_df.duplicated().sum())


9983
53079


In [16]:
### Checking the duplicates
duplicates=wine_review1_df[wine_review1_df.duplicated(subset=('title'))]
duplicates_sorted = duplicates.sort_values(by=['title'])
#duplicates_sorted
#should not be considered as duplicates - cleaning the data first


duplicates=wine_review2_df[wine_review2_df.duplicated(subset=('designation'))]
duplicates_sorted = duplicates.sort_values(by=['designation'])
#duplicates_sorted
#need to clean the data to interpretate the duplicates


### Missing data 

In [17]:
#Overall missing data in wine_review1_df dataset
wine_review1_df.isna().sum(),wine_review2_df.isna().sum()


(country           63
 description        0
 designation    37465
 points             0
 price           8996
 province          63
 region_1       21247
 region_2       79460
 title              0
 variety            1
 winery             0
 dtype: int64,
 country            5
 description        0
 designation    45735
 points             0
 price          13695
 province           5
 region_1       25060
 region_2       89977
 variety            0
 winery             0
 dtype: int64)

#### Variety variable 
- Removed the only instance with missing variety in wine_review1

In [18]:
#Variety missing data
wine_review1_df[wine_review1_df['variety'].isna()]
#Only one record is missing in wine_review1_df and most of the data for this wine is missing - Decided to remove the record
wine_review1_df = wine_review1_df.dropna(subset=['variety'])


#### Designation variable

In [19]:
m1=wine_review1_df['designation'].isna().sum()
m2=wine_review2_df['designation'].isna().sum()
print(f'Designation missing in wine_review1_df:{m1},in wine_review2_df:{m2}')


Designation missing in wine_review1_df:37464,in wine_review2_df:45735


##### Wine_review1_df

In [20]:
#Filling the missing designation using title variable(not displayed in the wine_review2_df dataset)

wine_review1_df['designation'].fillna(wine_review1_df['title'], inplace=True)
def remove_year(designation):
    pattern = r'\b\d{4}\b'  # Regex pattern to match four-digit year
    return re.sub(pattern, '', designation).strip()

# Remove year from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_year)

#Dealing with the parentheses values in designation
def remove_parentheses_text(designation):
    pattern = r'\s*\([^)]*\)'  # Regex pattern to match text within parentheses
    return re.sub(pattern, '', designation).strip()

# Remove text within parentheses from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_parentheses_text)


##### Wine_review2_df

In [21]:
#As no title provide the designation of the wine the missing designation in wine_review2_df must be dropped
wine_review2_df.dropna(subset='designation', inplace=True)


In [22]:
wine_review1_df.isna().sum()


country           63
description        0
designation        0
points             0
price           8996
province          63
region_1       21246
region_2       79459
title              0
variety            0
winery             0
dtype: int64

#### Region variable
Analysing the region_1 and region_2 variables and missing data:

##### Wine_review1_df

In [23]:
#Analysing the region_1 and region_2 variables
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review1_df['region_1'].fillna(wine_review1_df['region_2'], inplace=True)
wine_review1_df['region_1'].isna().sum()


21246

In [24]:
#Removing the region_2 column from the dataset
wine_review1_df.drop(columns='region_2', inplace=True)


In [25]:
wine_review1_df.isna().sum()


country           63
description        0
designation        0
points             0
price           8996
province          63
region_1       21246
title              0
variety            0
winery             0
dtype: int64

#### Title column 

In [26]:
#import re #import re module for regular expressions in the Import section
#filling the missing region with the region displayed in the title

#Function to extract the content within parentheses from title column
def extract(s):
    if isinstance(s, str):
        extracted = re.search(r'\((.*?)\)', s)
        if extracted:
            return extracted.group(1)
    return None
wine_review1_df['region_1'].fillna(wine_review1_df['title'].apply(extract), inplace=True)

wine_review1_df[wine_review1_df['region_1'].isna()]

#412 instances don't have a region displayed but have other useful data available


,country,description,designation,points,price,province,region_1,title,variety,winery
261,Argentina,"A blend of 50% Malbec with Cabernet, Syrah and...",Estate Blend Gran Reserva,89,37.0,Mendoza Province,None,Algodon 2008 Estate Blend Gran Reserva Red,Red Blend,Algodon
594,France,This medium-bodied blend of 50% Grenache and 5...,La Decelle,89,20.0,Rhône Valley,None,Lavau 2015 La Decelle Red,Rhône-style Red Blend,Lavau
622,US,"Mostly Syrah, with just a dash of Petite Sirah...",Sidewinder,87,25.0,Idaho,None,Coiled 2009 Sidewinder Red,Red Blend,Coiled
623,US,"This unusual blend of 85% Syrah, 10% Petit Ver...",Black Mamba,87,28.0,Idaho,None,Coiled 2010 Black Mamba Red,Red Blend,Coiled
913,NaN,"Amber in color, this wine has aromas of peach ...",Asureti Valley,87,30.0,NaN,None,Gotsa Family Wines 2014 Asureti Valley Chinuri,Chinuri,Gotsa Family Wines
...,...,...,...,...,...,...,...,...,...,...
129297,Spain,Jammy black-fruit aromas push up against raisi...,Payoya Negra,90,33.0,Andalucia,None,Finca la Melonera 2014 Payoya Negra Red,Red Blend,Finca la Melonera
129407,NaN,Dry spicy aromas of dusty plum and tomato add ...,Reserve,89,22.0,NaN,None,El Capricho 2015 Reserve Cabernet Sauvignon,Cabernet Sauvignon,El Capricho
129408,NaN,El Capricho is one of Uruguay's more consisten...,Reserve,89,22.0,NaN,None,El Capricho 2015 Reserve Tempranillo,Tempranillo,El Capricho
129590,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,None,Büyülübağ 2012 Shah Red,Red Blend,Büyülübağ


- Filling the missing region_1 data when applicable using title

In [27]:
#Trying to fill the missing province with the title information

def extract_province_from_title(title):
    pattern = r'\((.*?)\)'  # Regex pattern to match text within parentheses
    match = re.search(pattern, title)
    if match:
        return match.group(1)
    else:
        return None

# Extract province from 'Title' column
wine_review1_df['extracted_Province'] = wine_review1_df['title'].apply(extract_province_from_title)

# Fill missing 'Province' values with extracted province information
wine_review1_df['province'].fillna(wine_review1_df['extracted_Province'], inplace=True)
wine_review1_df.drop(columns=['extracted_Province'], inplace=True)


#checking the data that have both missing province and region
check = wine_review1_df['region_1'].isnull() & wine_review1_df['province'].isnull()
# Filter the DataFrame based on the condition
rows_missing_region_province = wine_review1_df[check]
#rows_missing_region_province

wine_review1_df['region_1'].fillna(wine_review1_df['province'], inplace=True)


In [28]:
wine_review1_df['region_1'].isna().sum()


63

In [29]:
#wine_review1_df[wine_review1_df['region_1'].isna()]
#dropping the 63 remaining instances that don't have country , province and region filled
wine_review1_df.dropna(subset='region_1', inplace=True)


##### Wine_review2_df

In [30]:
#Analysing the region_1 and region_2 variables
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review2_df['region_1'].fillna(wine_review2_df['region_2'], inplace=True)
wine_review2_df['region_1'].isna().sum()


19581

In [31]:
#Removing the region_2 column from the dataset
wine_review2_df.drop(columns='region_2', inplace=True)


In [32]:

wine_review2_df['region_1'].fillna(wine_review2_df['province'], inplace=True)
wine_review2_df[wine_review2_df['region_1'].isna()]
wine_review2_df.dropna(subset='region_1', inplace=True)


#### Province variable

##### Wine_review1_df

In [33]:
#Displaying the missing data
wine_review1_df[wine_review1_df['province'].isna()]
#wine_review1_df.shape[0]


,country,description,designation,points,price,province,region_1,title,variety,winery


In [34]:
wine_review1_df.isna().sum()


country           0
description       0
designation       0
points            0
price          8992
province          0
region_1          0
title             0
variety           0
winery            0
dtype: int64

##### Wine_review2_df

In [35]:
wine_review2_df.isna().sum()


country            0
description        0
designation        0
points             0
price          10271
province           0
region_1           0
variety            0
winery             0
dtype: int64

#### Price variable
Decision taken : Missing price filled with mean price of wines with same variety

##### Wine_review1_df

In [36]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review1_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review1_df.groupby('variety')['price'].mean())
mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable

wine_review1_df = pd.merge(wine_review1_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review1_df['price'] = wine_review1_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [37]:
#Dropping the mean_price created column to fill the NA price
wine_review1_df=wine_review1_df.drop(columns=['price_mean'])


In [38]:
#Round up the wine price
wine_review1_df['price']=wine_review1_df['price'].round(2)


In [39]:
#Dropping the last missing data if any
wine_review1_df.isna().sum()
wine_review1_df=wine_review1_df.dropna()


##### Wine_review2_df

In [40]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review2_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review2_df.groupby('variety')['price'].mean())
#mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable

wine_review2_df = pd.merge(wine_review1_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review2_df['price'] = wine_review1_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [41]:
wine_review2_df=wine_review2_df.drop(columns=['price_mean'])


In [42]:
wine_review2_df['price']=wine_review2_df['price'].round(2)


In [43]:
wine_review2_df=wine_review2_df.dropna()
wine_review2_df.isna().sum()


country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
title          0
variety        0
winery         0
dtype: int64

In [44]:
wine_review1_df.columns, wine_review2_df.columns


(Index(['country', 'description', 'designation', 'points', 'price', 'province',
        'region_1', 'title', 'variety', 'winery'],
       dtype='object'),
 Index(['country', 'description', 'designation', 'points', 'price', 'province',
        'region_1', 'title', 'variety', 'winery'],
       dtype='object'))

### Merging both datasets

In [45]:
wine_review1_df.isna().sum(), wine_review2_df.isna().sum()


(country        0
 description    0
 designation    0
 points         0
 price          0
 province       0
 region_1       0
 title          0
 variety        0
 winery         0
 dtype: int64,
 country        0
 description    0
 designation    0
 points         0
 price          0
 province       0
 region_1       0
 title          0
 variety        0
 winery         0
 dtype: int64)

In [46]:
wine_reviews_df=pd.concat([wine_review1_df, wine_review2_df])


In [47]:
wine_reviews_df.shape


(259781, 10)

In [48]:
wine_reviews_df.isna().sum()
#wine_reviews_df.head()


country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
title          0
variety        0
winery         0
dtype: int64

###  Outliers and anomalies on merged dataset



#### Numerical data : Identifying outliers 
- if >480k euros => most expensive bottle in the world
- if >100 for score

In [49]:
#Selecting the numerical data
numerical_columns = wine_reviews_df.select_dtypes(include=['float64', 'int64']).columns
numerical_columns


Index(['points', 'price'], dtype='object')

In [50]:
# Finding outliers
threshold_point=100
threshold_price=480000
outliers_point = wine_reviews_df[(abs(wine_reviews_df['points']) > threshold_point)]
outliers_price = wine_reviews_df[(abs(wine_reviews_df['price'])> threshold_price)]


In [51]:
#Displaying the outliers
outliers_point
outliers_price
#No price or points outliers


,country,description,designation,points,price,province,region_1,title,variety,winery


#### Categorical variables

##### Cleaning up the empty spaces in 'Designation' variable

In [52]:
missing_indices = wine_reviews_df[wine_reviews_df['designation'] == ''].index.tolist()

# Iterate over missing_indices and update 'designation' column
for i in missing_indices:
    # Split the title string into parts using space ' ' as a separator
    title_parts = wine_reviews_df['title'].iloc[i].split(' ')
    year_index = None
    series_index = None

    # Find the indices of the year and series numbers
    for j, part in enumerate(title_parts):
        if part.isdigit():
            if len(part) == 4:  # Assuming year has 4 digits
                year_index = j
            else:
                series_index = j

    # Extract designation before the year (and series if present)
    if year_index is not None:
        designation_parts = title_parts[:year_index]
        if series_index is not None:
            designation_parts.append(title_parts[series_index])
        designation = ' '.join(designation_parts)
    else:
        designation = wine_reviews_df.at[i, 'title']

    # Update the 'designation' column in the DataFrame
    wine_reviews_df.at[i, 'designation'] = designation


In [53]:
wine_reviews_df[wine_reviews_df['designation']==''].count()


country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
title          0
variety        0
winery         0
dtype: int64

##### Other Categorical variables : Identifying outliers 

In [54]:
import stat

#wine_reviews_df.info()
categorical_columns=wine_reviews_df.select_dtypes(include='object').columns


In [55]:
categorical_columns


Index(['country', 'description', 'designation', 'province', 'region_1',
       'title', 'variety', 'winery'],
      dtype='object')

##### Country

In [56]:
# Set max rows to display to a high number to show all unique values
pd.set_option('display.max_rows', None)

#wine_reviews_df['country'].value_counts()
#No issue


##### Designation and variety
- Both columns should be merged as they provide better understanding of the wine variety together - new variable created

In [57]:
!pip install unidecode
from unidecode import unidecode  # Import unidecode library to preserve accents

wine_reviews_df['designation'].value_counts()
#Designation column has false values and is an additional information given by the wine variety variable
#decision merging both column as unique wine variety variable
wine_reviews_df['wine_variety']=wine_reviews_df['designation']+' '+wine_reviews_df['variety']

#Removing parentheses or other punctuations from the column wine_variety

def remove_non_year_numbers(text):
    # Remove numbers that are not likely to represent years
    cleaned_text = re.sub(r'\b(?<!\d)(?!(\d{4}|\d{2}))\d+\b', '', text)
    return cleaned_text

def clean_categorical_column(df, column_name):
    # Apply the remove_non_year_numbers function to the specified column
    df[column_name] = df[column_name].astype(str).apply(remove_non_year_numbers)
    return df

#Cleaning the variable wine_variety variable - Removing the repeated words in the variable
def remove_repeated_words(text):
    sentences = text.split('. ')
    cleaned_sentences = []

    for sentence in sentences:
        words = sentence.split()
        unique_words = []
        seen_words = set()

        for word in words:
            # Remove non-alphabetic characters, excluding the year or series of number that could be a year
            clean_word = re.sub(r'[^a-zA-Z\s]|(?<!\d)\d{1,2}(?!\d)', '', word)

            if clean_word and clean_word not in seen_words:
                unique_words.append(clean_word)
                seen_words.add(clean_word)

        cleaned_sentences.append(' '.join(unique_words))

    return '. '.join(cleaned_sentences)


In [58]:
#Cleaning all categorical variables the same way than wine_variety
cat_col=['country', 'description', 'designation', 'province', 'region_1',
       'title', 'variety', 'winery','wine_variety']
for col in cat_col:
    wine_reviews_df[col] = wine_reviews_df[col].apply(remove_repeated_words)
#Removing parentheses or other punctuations from the column wine_variety
wine_reviews_df = clean_categorical_column(wine_reviews_df, 'wine_variety')


#### CLeaning up the designation that are Ros instead of Rose

In [59]:


wine_reviews_df.loc[wine_reviews_df['designation'] == 'Ros', 'designation'] = 'Rose'
wine_reviews_df.loc[wine_reviews_df['variety'] == 'Ros', 'variety'] = 'Rose'
#wine_reviews_df[wine_reviews_df['designation']=='Ros']


#### Updating the empty winery when applicable 

In [60]:
# Identifying the empty winery variable
wine_reviews_df[wine_reviews_df['winery']=='']
#Updating the NV winery when applicable


# Check rows where 'designation' is 'Ros' and 'title' contains 'NV'
selected_rows = wine_reviews_df[(wine_reviews_df['winery'] == '') & (wine_reviews_df['title'].str.contains('NV '))]
selected_rows
# Update 'winery' to 'NV' where 'designation' is 'Ros' and 'title' contains 'NV'
wine_reviews_df.loc[(wine_reviews_df['winery'] == '') & (wine_reviews_df['title'].str.contains('NV')), 'winery'] = 'NV'

# Delete the ones that don't have winery displayed
wine_reviews_df = wine_reviews_df[wine_reviews_df['winery'] != '']

# Check rows where 'winery' is empty to verify the changes
wine_reviews_df[wine_reviews_df['winery'] == '']


,country,description,designation,points,price,province,region_1,title,variety,winery,wine_variety


##### Cleaning up the variety with errors 

In [61]:
replacements = {
    'Gewrztraminer': 'Gewürztraminer',
    'Gewurztraminer': 'Gewürztraminer',
    'Ros': 'Rose',
    'Carmenre': 'Carmenère',
    'Albario': 'Albariño',
    'Roseeeato': 'Rosato',
    'Roseeato': 'Rosato',
    'Torronts': 'Torrontes',
    'Roseee': 'Rose',
    'Rosee': 'Rose',
    'Blaufrnkisch': 'Blaufränkisch',
    'Menca': 'Mencia',
    'Smillon': 'Semillon',
    'Mourvdre': 'Mourvedre',
    'Anto Vaz': 'Antao Vaz',
    'G-S-M': 'GSM',
    'St. Laurent': 'St Laurent',
    'Xarel-lo': 'Xarello'
}

wine_reviews_df['variety'] = wine_reviews_df['variety'].replace(replacements, regex=True)


##### Cleaning up the wine_variety with errors

In [62]:
# Replace wrong wine_variety in the 'wine_variety' column
replacements = {
    'Gewrztraminer': 'Gewürztraminer',
    'Gewurztraminer': 'Gewürztraminer',
    'Ros': 'Rose',
    'Carmenre': 'Carmenère',
    'Albario': 'Albariño',
    'Roseeeato': 'Rosato',
    'Roseeato': 'Rosato',
    'Torronts': 'Torrontes',
    'Roseee': 'Rose',
    'Rosee': 'Rose',
    'Blaufrnkisch': 'Blaufränkisch',
    'Menca': 'Mencia',
    'Smillon': 'Semillon',
    'Mourvdre': 'Mourvedre',
    'Anto Vaz': 'Antao Vaz',
    'G-S-M': 'GSM',
    'St. Laurent': 'St Laurent',
    'Xarel-lo': 'Xarello',
    'Aligot':'Aligoté',
    'Frburgunder':'Fruhburgunder',
    'Sptburgunder':'Spatburgunder',
    'Ximnez':'Ximenez',
    'MllerThurgau':'Müller-Thurgau',
    'Castelo':'Castello',
    'Ruch':'Ruché',
    'Carinea':'Carinena',
    'Souso':'Sousao',
    'Regal':'Regala',
    'Valdigui':'Valdiguié',
    'Aragons':'Aragonez',
    'Jacqure':'Jacquère',
    'Pas':'Pass',
    'Trockenbeerenaulese Smling':'Trockenbeerenauslese Smiling',
    'Trockenbeerenauslese Smling':'Trockenbeerenauslese Smiling',
    'Hrslevel':'Hárslevelű',
    'Alvarelho':'Alvarelhao',
    'Cdega do Larinho':'Codega do Larinho',
    'Graevina':'Grasevina',
    'Boazkere':'Bogazkere',
    'Romneasc':'Romaneasca',
    'Forcall':'Forcallat',
    "Babi":'Babic',
    'ilavka':'Zilavka',
    'Rnsk':'Rynsky',
    'St. Vincent':'St Vincent'
    }

wine_reviews_df['wine_variety'] = wine_reviews_df['wine_variety'].replace(replacements, regex=True)


#####  Province

In [63]:
#wine_reviews_df['province'].value_counts()
# No outliers or anonalies found


In [64]:
#wine_reviews_df.head()


In [65]:
#wine_reviews_df['winery'].value_counts()
# No winery outliers or anomalies found



### Create wine type dictionary

In [66]:
unique_variety=wine_reviews_df['wine_variety'].unique()
#pd.set_option('display.max_rows', None)
#The wine type dictionary can't be only based on the variety as some grape variety can actually result to red, white and rose wines
#Decided to use the created variable wine_variety = designation+variety to build the dictionary
#pd.DataFrame(wine_reviews_df['wine_variety'].value_counts())


In [67]:
#Create .json files of all wine varieties / type to be called in the function

import json

# Define lists of varieties
red_varieties = ['dAbruzzo','Bridão Touriga Nacional','Pinot Noir', 'Red Blend', 'Merlot', 'Syrah', 'Zinfandel',
                     'Malbec', 'Nero', 'Noir', 'Nebbiolo', 'Sangiovese', 'Red','red' 'Tempranillo','Rosso',' Gamay','Gamay'
                     ' Shiraz','Shiraz', 'Sirah', 'Sangiovese Grosso', 'Barbera','Port', 'Grenache', 'Corvina','Rondinella','Molinara',
                     'Tempranillo', 'Carmenère', 'Aglianico', 'Garnacha','Cannubi Nebbiolo', 'Estate Mourvèdre', 'Grenache',
                     'Classic Zweigelt', 'Reserve St Laurent', 'Passito Sagrantino','Belsito Frappato','Quiévremont Meritage','Nero',
                     'Sallier de la Tour Inzolia','I Tratturi Primitivo','Petit Verdot','Dolia Monica','Bridão Touriga Nacional'
                     'Graciano','TannatCabernet','Riserva Prugnolo Gentile','Gesture GSM','Red','Bonarda',
                     'Blaufränkisch','Cabernet',' Tannat','Cannonau','Mencía','Sagrantino','Brezza Dolcetto','Graciano','Negroamaro',
                     'Magnificat Meritage', 'Eszencia Furmint', 'Zweigelt', 'Caverio GSM','Dolcetto','Pinotage',
                    'Früburgunder','Sousão','Cinsault','Primitivo','Mascalese','Zweigelt', 'Noces de Feu Duras','Meritage',
                    'Papaskarasi','Charbono','Cream Sherry', 'GSM', 'Lavender Ridge Mourvèdre','Pinot Meunier',
                    'Teroldego','Claret','Montepulciano','Touriga Nacional','Tinta','Aragonez','red blend','Lagrein','Monastrell',
                    'Alicante Bouschet','Chambourcin','Mourvèdre','Malvasia', 'Selección Especial 1 Moscatel','Pugnitello', 'Sherry',
                     'Colheita Moscatel', 'Old Vines 3 Moscatel','Maturana','Neagra', 'Cunto Pallagrello','Agiorgitiko','Frappato',' St Laurent',
                    'Alfrocheiro','Carignano','Magliocco',' Saperavi','Cesanese','Tinto','Uva di Troia', 'Bethlehem Valley Norton',
                     'Dry Norton', 'The Full Nelson Norton', 'Estates Norton', 'Augusta Winery Norton', 'Reserve Norton',
                     'Limited Release Norton', 'Estate Bottled Norton', 'Molliver Vineyards Norton','Refosco','Manzoni','Ruché','Xinomavro','Petite Verdot',
                    'Spätburgunder','Baga','Lemberger','Dornfelder','Perricone','Ximénez','Aleatico',' Marselan','Schiava','Lemberger',
                     'Carignan','Gentile','Valdiguié','Mondeuse','Negrette','Trincadeira','Touriga','Prieto Picudo','Gaglioppo','Piedirosso'
                    'Susumaniello','Carignane','Counoise','Carcajolu','Madeira','Kalecik Karasi','Karasakiz','País','Marzemino','Castelão',
                    'Teran','Trepat',' Freisa','Argaman','Bobal','Piedirosso','Mansois','Susumaniello','Castelão','Cappuccio','Argaman','Mazuelo',
                    'Vranec','Mavrud','Forcallà','Aragonês','Jaen','Argaman','Prunelard','Grignolino',' Mataro','Groppello','Raboso',
                    'Alvarelhão','Poulsard','Monica','Carineña','Carignan','Ciliegiolo','Sciaccerellu','Rufete','Alicante','Mavroudi',
                    'Casavecchia','Mantonico','Trousseau','Rebo','Vespolina',' Kekfrankos','Braucol','Marquette','Blauburgunder',
                    'Kekfrankos','Fer','Servadou','Fer Servadou','Duras','Okuzgozu','Bual','Kadarka','Cabernet Sauvignon','Sauvignon Cabernet',
                    'Franc','Carignan','Cinsault','Grenache','Mourvèdre','Pinot noir','Sangiovese','Mission','Sankt Laurent','Abouriou','Nielluciu',
                    'Vinhão','Souzao','St Laurent','Moscatel do Douro','Orangetraube','Negro','Grolleau','Canaiolo','Mavrotragano','Bovale',
                    'Frankovka','Nielluciu','Gragnano','Mavrodaphne','Schwartzriesling','Bastardo','40 Years Moscatel','Vidadillo','Vinhão',
                    'Zigu','Abouriou','Canaiolo','Norton','Bastardo','Babić','Mandilaria','Albarossa','Babić','Monukka','Uvalino',
                    'Boğazkere','Chelois','Durif','10 Anos Moscatel','Sirica','Blatina','Karalahna','Feteasca','St Vincent','Cherry','Colorino',
                    ' Blauburger','Centesimino','Ramisco','Catalanesca','Especial Moscatel','10YearsOld Moscatel','Gamza','Kadarka','Mavrokalavryta',
                    'Tintilia','Kotsifali','Parraleta','Trollinger','Kuntra','Feteasca','Sellium Moscatel','Pignolo','Pignol','Trollinger','Blaufränkisch'
                    'Mencia','Mourvedre',' Mencia','Fruhburgunder','Spatburgunder','Ruché','Carinena','Sousao','Valdiguié','Valdiguie','Jacquère','Alvarelhao',
                    'Bogazkere','Forcallat','Babic']

white_varieties = ['Muscat','Muscadelle','Chardonnay', 'Riesling', 'Pinot Gris', 'Sauvignon Blanc', 'Pinot Grigio', 'Gewürztraminer', 'Bianco',
                       'White', 'Gris', 'Blanc', 'Viognier', 'Pinot Gris', 'Feudo Montoni Catarratto', 'Grillo',
                       'Chablis', 'Spyro Albariño', 'La Réserve Petit Manseng', 'Grüner Veltliner',
                       'White Blend', 'Glera', 'Chenin Blanc', 'Albariño', 'Pinot Blanc','Sur Lie Melon', 'Grüner Veltliner',
                       'Torrontés', 'Chenin Blanc', 'Col Fondo Glera','Vermentino','white','De Silva Sauvignon','Megyer Dry Furmint',
                       'Felluga Friulano','Assyrtico','Massovivo Vermentino',' Garganega','Caldaro Sauvignon','Tannenberg Sauvignon',
                       'Gewürztraminer','Jaune','Verdejo','Verdelho','Verduzzo','Sémillon','Vernaccia','Verdicchio',' Verdicchio','Greco di Tufo'
                       'Lucas Vineyards Vignoles', 'Franz Haas Sauvignon', 'Lage Modler Zierfandler', 'Gold Label Cortese',
                       'Reserve Icewine Vidal', 'Vigneti Le Monde Sauvignon', 'Doux Sawyer Lindquist Vineyard Marsanne',
                       'Scheurebe', 'Kerner','Vilana', 'Le Marne Cortese', 'Viura', 'Sturm Sauvignon','Veltliner', 'Antão Vaz',
                      'Viura','Silvaner','Colombard','Sylvaner','Fiano','Roussanne', 'Goru Verde Monastrell','Xarello','Trebbiano',
                       'Cubia Tenuta Ficuzza Insolia', 'Vinai dellAbbate Ribolla Gialla', 'Colutta Friulano','Weissburgunder',
                       'Spätlese Traminer','B Minor MarsanneRoussanne','Silvaner', 'Cusumano Insolia','Greco Tufo', 'Cembra Sauvignon',
                       'Barraco Zibibbo','Falanghina', 'Tercic Ribolla Gialla','MüllerThurgau','Kerner','Greco di Tufo',
                       'Nals Margreid Sauvignon','Petit Manseng','Muskat Ottonel','Muscat Ottonel','Marsanne','Bical','Nals Margreid Sauvignon',
                       'Doña Beatriz Sauvignon', 'Urra di Mare Sauvignon', 'Ronco dei Tassi Sauvignon', 'Vigna Maso Tratta Sauvignon', 'Quarz Sauvignon',
                       'Conte dAttimisManiago Sauvignon', 'Kirchleiten Sauvignon', 'Passion Sauvignon', 'CaRonesca Sauvignon', 'Classic Sauvignon',
                       'Petrucco Sauvignon', 'Russiz Superiore Sauvignon', 'Naun Sauvignon', 'Winkl Sauvignon', 'Lafòa Sauvignon',
                       'Weingut Niklas Sauvignon', 'Castel Giovanelli Sauvignon', 'Jermann Sauvignon', 'Masut da Rive Sauvignon', 'Solamini et Amicis Passito Sauvignon',
                       'Mission St Vincent Sauvignon', 'Vieris Sauvignon', 'Col Matìss Sauvignon', 'Castel Ringberg Sauvignon', 'La Rajade Sauvignon', 'De La Tour Sauvignon',
                       'Sulz Sauvignon', 'Castelvecchio Sauvignon', 'Riserva Sauvignon', 'Gradisciutta Sauvignon', 'Stern Sauvignon', 'Aquilis Sauvignon', 'La Tunella Sauvignon',
                       'Prail Sauvignon', 'Ferruccio Sgubin Sauvignon', 'Primosic Sauvignon', 'Vigne del Borgo Sauvignon', 'DAmblè Sauvignon', 'Caligo Sauvignon', 'Mario Schiopetto Sauvignon',
                       'Plozner Sauvignon', 'Karneid Sauvignon','Turbiana','Bianca','Moscadello','Traminer','Assyrtiko','Muskateller','Welschriesling','Pecorino',
                      'Alvarinho','Albanello','Villa Russiz Sauvignon', 'Bovin Sauvignon', 'Indra Sauvignon', 'Abbazia di Novacella Sauvignon', 'Kofl Sauvignon', 'Piere Sauvignon',
                       'Ferrata Sauvignon', 'Sanct Valentin Sauvignon', 'Reina de Castilla Sauvignon', 'Soliloquy Sauvignon Musqué', 'Bricco Rovella Sauvignon', 'Mejo Sauvignon',
                       'Turmhof Sauvignon', 'Segrè Sauvignon', 'Livio Felluga Sauvignon', 'Terre Magre Sauvignon', 'Floreado Sauvignon', 'Borgo del Tiglio Sauvignon',
                       'Rive Alte Sottomonte Sauvignon', 'Kössler Sauvignon', 'Valbuins Sauvignon', 'Attems Sauvignon', 'Colutta Sauvignon', 'Cormòns Sauvignon', 'Cicinis Sauvignon',
                       'Montespina Sauvignon', 'Praesulis Sauvignon', 'Fumat Sauvignon', 'Zorzettig Sauvignon','Picpoul de Pinet','Cortese','Melon','Godello','Friulano',
                       'Viña Esmeralda Moscatel','MR Mountain Wine Moscatel','Botani Old Vines Moscatel','Vino Dulce Natural Moscatel','Vino Dulce Moscatel',
                       'Moscatel de la Marina', 'El Bosque Moscatel','Selección Especial 1 Moscatel', 'Esencia de Almachar Moscatel', 'Late Harvest Moscatel',
                       'Miguel Asensio Vino Dulce de Moscatel', 'Victoria 2 Moscatel', 'Vendimia Tardia Single Vineyard Moscatel', 'Casta Diva Cosecha Miel Dulce Moscatel',
                       'Mesta MacabeoMoscatel', 'Quinta da Arrancosa Moscatel Graúdo','Picpoul','Moschofilero','Arinto', 'Caiatì Pallagrello', 'Fontanavigna Pallagrello',
                       'Acquavigna Pallagrello', 'Beerenauslese Rieslaner','Zierfandler', 'Catarratto','Muskat','Greco','Grechetto','Albana','Tocai',
                      'Savagnin','Gialla','Nosiola','Encruzado','Carricante', 'Grotta del Sole Asprinio di Aversa','Lucas Vineyards Vignoles',
                      'Inzolia','Insolia','Malagousia','Malagouzia','Cerceal','Pires','Rotgipfler','Yapincak','Edelzwicker','Siria','Morava','Passerina','Loureiro',
                      'Posip','Regalǎ','Furmint','Vidal','Insolia','Auxerrois','Aligoté','Arneis','Mali','Altesse','Kisi','Palomino',
                      'Nascetta','Tokaji','Symphony','Durella','Clairette','Chasselas','Siegerrebe','Manseng','Morillon','Zibibbo','Torbato','Cercial',
                      'Kangoun','Jacquère','Mauzac','Diamond','Neuburger','Sämling','Hárslevelü','Rkatsiteli','Jacquère','Mauzac','Picolit',
                      'Sacy','Mauzac','Vitovska','Rkatsiteli','Grauburgunder','Vignoles','Grigio','Debit','Verdil','Treixadura','Loin de',
                      'Neuburger','Tokay','Jacquère','Timorasso','Pigato','Macabeo','Marawi','Irsai Oliver','Traminette','Pignoletto',
                      'Mantonico','Avesso','Emir','Narince','Grecanico','Angevine','Avesso','Zuri','Hondarrabi Zuri','Nuragus','Xynisteri',
                      'Avesso','Roditis','Plyto','Coda di Volpe','Rivaner','Müller-Thurgau','Tamjanika','Narince','Rebula',' Bual',
                      'Grecanico','Savatiano',' Roditis','Romorantin','Cococciola','Thrapsathiri','Mantonico','Graševina','Rivaner',
                      'Narince','Mtsvane','Códega do Larinho','Sideritis','Azal','Sauvignon','Viosinho','Paralleda','Maria Gomes','Fernão Pires',
                      'Airen','Erbaluce','Verdosilla', 'Naturalmente Dulce Moscatel','Vespaiolo','Aidani','Rolle','Žilavka','Estate Muscadine',
                      'Rolle','Zlahtina','Gouveio','Elbling','Dorada Seco Moscatel','Charmat Moscatel','Pallagrello','Misket','Rieslaner',
                      'Alejandría','Robola','Graúdo','Verdeca','Nasco','Athiri','Cayuga','Tamianka','Rabigato','Meseguera','Merseguera','Xinisteri',
                      'Rýnský','Malvar','Românească','Chardonel','Plageoles Muscadel','Ansonica','Apple','Asprinio','Sercial','Premsal',
                      'Jampal','Picapoll','Moscatel Seco','Trajadura','Chinuri','Favorita','Dafni','Petit Courbu','Old Vines No 3 Moscatel','Torontel',
                      'Molino Real Mountain Wine Moscatel','Molino Real Moscatel','Asprinio','Emblema Caprettone','Ondenc','Torrontes','Semillon','Antao Vaz','Xarello',
                      'Pedro Ximenez','Castello','Moscatel','Regala','Pass','Trockenbeerenauslese Smiling','Hárslevelű','Codega do Larinho',
                  'Grasevina','Vinho Verde','Vinho','Romaneasca','Zilavka','Rynsky']

rose_varieties = ['Rose', 'Rosé', 'Rosato','Rosè','Alambre Moscatel',  'Colheita Moscatel','Clã Moscatel', 'Colecção Privada Moscatel Roxo', 'José Maria da Fonseca Moscatel de Setúbal',
                     'Rosado', 'Moscatel de Setubal', 'Loboroxo Moscatel Roxo','rosé','Branco Moscatel',]

sparkling_varieties = ['Champagne', 'Prosecco', 'Cava', 'Sparkling', 'Champagne Blend', 'Moscato', 'Cave Amadeu Moscatel', 'Tarima Moscatel',
                          'Extra Dry Prosecco', 'Dry Glera' ,'Bruto Avesso', 'Anni Venti Metodo Classico Greco','Lambrusco', 'Casa Perini NV Moscatel',
                          'Brachetto','Pignoletto','Andino Moscatel',]

red_varieties = list(set(red_varieties))
white_varieties=list(set(white_varieties))
rose_varieties=list(set(rose_varieties))

# Create a dictionary to store all lists
varieties_dict = {
    "red_varieties": red_varieties,
    "white_varieties": white_varieties,
    "rose_varieties": rose_varieties,
    "sparkling_varieties": sparkling_varieties
}

# json_path = os.path.expanduser('~/sabrinaauger/code/sabrinaauger/wino/data/json/wine_varieties.json')
# Save the dictionary to a JSON file
with open('/home/jackzhong12/code/sabrinaauger/wino/data/json/wine_varieties.json', 'w') as json_file:
    json.dump(varieties_dict, json_file)


In [68]:
variety_to_type = {}

# Load the JSON file containing wine varieties
with open('/home/jackzhong12/code/sabrinaauger/wino/data/json/wine_varieties.json', 'r') as json_file:
    varieties_dict = json.load(json_file)

# Extract different types of wine varieties from the loaded dictionary


def assign_type(variety):

    red_varieties = varieties_dict["red_varieties"]
    white_varieties = varieties_dict["white_varieties"]
    rose_varieties = varieties_dict["rose_varieties"]
    sparkling_varieties = varieties_dict["sparkling_varieties"]

    variety_lower = variety
    red_varieties = [var for var in red_varieties]
    white_varieties = [var for var in white_varieties]
    rose_varieties = [var for var in rose_varieties]
    sparkling_varieties = [var for var in sparkling_varieties]

    if any(var in variety for var in white_varieties):
        return 'White'
    elif any(var in variety for var in red_varieties):
        return 'Red'
    elif any(var in variety for var in rose_varieties):
        return 'Rosé'
    elif any(var in variety for var in sparkling_varieties):
        return 'Sparkling'
    else:
        return 'Other'

# Assuming wine_reviews_df is your DataFrame and 'wine_variety' is the column with wine varieties
for variety in wine_reviews_df['wine_variety']:
    # Manually assign types based on the variety and on the logic built in the assign_type function
    variety_to_type[variety] = assign_type(variety)


In [69]:
#variety_to_type


In [70]:
#Display the remaining wines without a
other_varieties = [variety for variety, wine_type in variety_to_type.items() if wine_type == 'Other']
other_varieties


[]

In [71]:
len(other_varieties)


0

In [72]:
wine_reviews_df['wine_type']=wine_reviews_df['wine_variety'].map(variety_to_type)


### Create .csv file of cleaned data

In [73]:

wine_reviews_df.to_csv('~/code/sabrinaauger/wino/data/cleaned_data/wine_reviews.csv', index=False)
test=pd.read_csv('~/code/sabrinaauger/wino/data/cleaned_data/wine_reviews.csv')
#test.isna().sum()


### Preprocessing description

In [74]:


# Function for text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize the text
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize words
    return ' '.join(tokens)

# Apply preprocessing to the description column
wine_reviews_df['description']=wine_reviews_df['description'].apply(preprocess_text)


In [75]:
wine_reviews_df.head()


,country,description,designation,points,price,province,region_1,title,variety,winery,wine_variety,wine_type
0,Italy,aroma include tropical fruit broom brimstone d...,Vulk Bianco,87,23.24,Sicily Sardinia,Etna,Nicosia Vulk Bianco Etna,White Blend,Nicosia,Vulk Bianco White Blend,White
1,Portugal,ripe fruity wine smooth still structured firm ...,Avidagos,87,15.00,Douro,Douro,Quinta dos Avidagos Red Douro,Portuguese Red,Quinta dos Avidagos,Avidagos Portuguese Red,Red
2,US,tart snappy flavor lime flesh rind dominate gr...,Rainstorm Pinot Gris,87,14.00,Oregon,Willamette Valley,Rainstorm Pinot Gris Willamette Valley,Pinot Gris,Rainstorm,Rainstorm Pinot Gris,White
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,87,13.00,Michigan,Lake Michigan Shore,St. Julian Reserve Late Harvest Riesling Lake ...,Riesling,St. Julian,Reserve Late Harvest Riesling,White
4,US,much like regular bottling come across rather ...,Vintners Reserve Wild Child Block,87,65.00,Oregon,Willamette Valley,Sweet Cheeks Vintners Reserve Wild Child Block...,Pinot Noir,Sweet Cheeks,Vintners Reserve Wild Child Block Pinot Noir,Red


### Create the dry/sweet variable 

In [76]:
!pip install gensim
from gensim.models import Word2Vec

# List of words related to sweetness
sweet_words = ['sweet', 'dessert', 'sweetness','creamy','sugary','sweetened','sugared','honeyed',
'candied',
'syrupy'
]

# Training data
training_data = [description.split() for description in wine_reviews_df['description']]

# Train the Word2Vec model
model = Word2Vec(sentences=training_data, vector_size=100, window=5, min_count=1, workers=4)

# Find similar words to each word in sweet_words
similar_words = {}
for word in sweet_words:
    similar_words[word] = [w for w, _ in model.wv.most_similar(word)]

print(similar_words)


{'sweet': ['sweetened', 'sweettasting', 'confected', 'sweetness', 'flavor', 'lush', 'luscious', 'bitterskin', 'jammy', 'jam'], 'dessert': ['moscato', 'passito', 'portstyle', 'ambercolored', 'classify', 'moscatobased', 'fruitbased', 'craze', 'goldencolored', 'limnos'], 'sweetness': ['sugar', 'sweet', 'creaminess', 'bitterness', 'ripeness', 'softness', 'carbon', 'roundness', 'cloying', 'fizzmake'], 'creamy': ['round', 'creaminess', 'buttery', 'rounded', 'cream', 'viscous', 'broad', 'custardy', 'oily', 'unctuous'], 'sugary': ['jellied', 'cloying', 'lifesaver', 'porty', 'gummy', 'sugared', 'candied', 'seemingly', 'confected', 'piefilling'], 'sweetened': ['caramelized', 'sugared', 'granola', 'enriched', 'cookie', 'sweet', 'lifesaver', 'coated', 'lemondrop', 'sprinkled'], 'sugared': ['jellied', 'sugary', 'sweetened', 'lifesaver', 'bar', 'confection', 'mashed', 'granola', 'chutney', 'jelly'], 'honeyed': ['honey', 'quince', 'apricot', 'mango', 'tropical', 'honeysuckle', 'unctuous', 'pineapple'

In [77]:
#sweet_words = ['sweet', 'dessert','muscat','muskat','moscato','ice wine','sweetness']

def check_sweetness(description):
    description_lower = description.lower()
    for word in similar_words:
        if word in description_lower:
            return 'sweet'
    return 'dry'

wine_reviews_df['Dry/Sweet'] = wine_reviews_df['description'].apply(check_sweetness)


In [78]:
#Ensuring that teh function with Spacy worked
wine_reviews_df['Dry/Sweet'].value_counts()


Dry/Sweet
dry      212519
sweet     47252
Name: count, dtype: int64

In [79]:
wine_reviews_df.head()


,country,description,designation,points,price,province,region_1,title,variety,winery,wine_variety,wine_type,Dry/Sweet
0,Italy,aroma include tropical fruit broom brimstone d...,Vulk Bianco,87,23.24,Sicily Sardinia,Etna,Nicosia Vulk Bianco Etna,White Blend,Nicosia,Vulk Bianco White Blend,White,dry
1,Portugal,ripe fruity wine smooth still structured firm ...,Avidagos,87,15.00,Douro,Douro,Quinta dos Avidagos Red Douro,Portuguese Red,Quinta dos Avidagos,Avidagos Portuguese Red,Red,dry
2,US,tart snappy flavor lime flesh rind dominate gr...,Rainstorm Pinot Gris,87,14.00,Oregon,Willamette Valley,Rainstorm Pinot Gris Willamette Valley,Pinot Gris,Rainstorm,Rainstorm Pinot Gris,White,dry
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,87,13.00,Michigan,Lake Michigan Shore,St. Julian Reserve Late Harvest Riesling Lake ...,Riesling,St. Julian,Reserve Late Harvest Riesling,White,dry
4,US,much like regular bottling come across rather ...,Vintners Reserve Wild Child Block,87,65.00,Oregon,Willamette Valley,Sweet Cheeks Vintners Reserve Wild Child Block...,Pinot Noir,Sweet Cheeks,Vintners Reserve Wild Child Block Pinot Noir,Red,dry


# Adding the aroma column in the dataset

In [80]:
# Specify the directory containing your JSON files
aromas_directory = os.path.expanduser('~/code/sabrinaauger/wino/interface/aromas')


In [81]:
json_files = [file for file in os.listdir(aromas_directory) if file.endswith('.json')]

# Read and store content of each file
file_contents = {}
for file_name in json_files:
    # Create the full file path by joining the directory and file name
    file_path = os.path.join(aromas_directory, file_name)

    with open(file_path, 'r') as file:
        # Load JSON content
        content = json.load(file)

        # Create variable name based on file name
        variable_name = file_name.split('.')[0] + '_word'

        # Store content in the dictionary with variable name as the key
        file_contents[variable_name] = content

# Extract aroma lists
fruity = file_contents.get('fruity_word')
floral = file_contents.get('floral_word')
earthy = file_contents.get('earthy_word')
herbal = file_contents.get('herbal_word')

# Training data
training_data = [description.split() for description in wine_reviews_df['description']]

# Train the Word2Vec model
model = Word2Vec(sentences=training_data, vector_size=100, window=5, min_count=1, workers=4)

# Create a dictionary to store similar words for each aroma
similar_aromas = {
    'fruity': [w for word in fruity if word in model.wv.key_to_index for w, _ in model.wv.most_similar(word)],
    'floral': [w for word in floral if word in model.wv.key_to_index for w, _ in model.wv.most_similar(word)],
    'earthy': [w for word in earthy if word in model.wv.key_to_index for w, _ in model.wv.most_similar(word)],
    'herbal': [w for word in herbal if word in model.wv.key_to_index for w, _ in model.wv.most_similar(word)],
}

fruity_words = similar_aromas['fruity']
floral_words = similar_aromas['floral']
herbal_words = similar_aromas['herbal']
earthy_words = similar_aromas['earthy']


def check_aromas(description):
    description_lower = description.lower()
    for word in fruity_words:
        if word in description_lower:
            return 'Fruity'
            break
    for word in floral_words:
            return 'Floral'
            break
    for word in herbal_words:
            return 'Herbal'
            break
    for word in earthy_words:
            return 'Earthy'
            break
    return None

wine_reviews_df['aroma'] = wine_reviews_df['description'].apply(check_aromas)


In [82]:
wine_reviews_df.head()


,country,description,designation,points,price,province,region_1,title,variety,winery,wine_variety,wine_type,Dry/Sweet,aroma
0,Italy,aroma include tropical fruit broom brimstone d...,Vulk Bianco,87,23.24,Sicily Sardinia,Etna,Nicosia Vulk Bianco Etna,White Blend,Nicosia,Vulk Bianco White Blend,White,dry,Fruity
1,Portugal,ripe fruity wine smooth still structured firm ...,Avidagos,87,15.00,Douro,Douro,Quinta dos Avidagos Red Douro,Portuguese Red,Quinta dos Avidagos,Avidagos Portuguese Red,Red,dry,Fruity
2,US,tart snappy flavor lime flesh rind dominate gr...,Rainstorm Pinot Gris,87,14.00,Oregon,Willamette Valley,Rainstorm Pinot Gris Willamette Valley,Pinot Gris,Rainstorm,Rainstorm Pinot Gris,White,dry,Fruity
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,87,13.00,Michigan,Lake Michigan Shore,St. Julian Reserve Late Harvest Riesling Lake ...,Riesling,St. Julian,Reserve Late Harvest Riesling,White,dry,Fruity
4,US,much like regular bottling come across rather ...,Vintners Reserve Wild Child Block,87,65.00,Oregon,Willamette Valley,Sweet Cheeks Vintners Reserve Wild Child Block...,Pinot Noir,Sweet Cheeks,Vintners Reserve Wild Child Block Pinot Noir,Red,dry,Floral


# Save cleaned wine_reviews_df dataset as .CSV

In [85]:
import os
import zipfile

# Directory path for the zipped file
directory_path = '~/code/sabrinaauger/wino/data/cleaned_data/'

# Ensure the directory exists or create it if it doesn't
os.makedirs(directory_path, exist_ok=True)


In [86]:
import zipfile


# Path for the zipped file
zip_file_path = '/Users/sabrinaauger/code/sabrinaauger/wino/data/cleaned_data/wine_reviews.zip'
csv_file_path ='/Users/sabrinaauger/code/sabrinaauger/wino/data/cleaned_data/wine_reviews.csv'
wine_reviews_df.to_csv(csv_file_path, index=False)

# Create a Zip file and add the CSV file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(csv_file_path, os.path.basename(csv_file_path))

# Reading the zipped file as a DataFrame
with zipfile.ZipFile(zip_file_path, 'r') as zipf:
    with zipf.open(os.path.basename(csv_file_path)) as myfile:
        test = pd.read_csv(myfile)


# Aroma Column creation

In [3]:
!python ~/code/sabrinaauger/wino/interface/aroma_column.py


Traceback (most recent call last):
  File "/home/jackzhong12/code/sabrinaauger/wino/interface/aroma_column.py", line 7, in <module>
    wine_reviews_df = pd.read_csv('~/code/sabrinaauger/wino/data/clean_data/test_wine_reviews_df.csv')
  File "/home/jackzhong12/.pyenv/versions/3.10.6/envs/wino/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 948, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/jackzhong12/.pyenv/versions/3.10.6/envs/wino/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 611, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/jackzhong12/.pyenv/versions/3.10.6/envs/wino/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1448, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/home/jackzhong12/.pyenv/versions/3.10.6/envs/wino/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1705, in _make_engine
    self.handles = get_handle(
  File "/home/j

# Modelisation
#Add Haiwei code

In [87]:
# Vectorization (e.g., TF-IDF, CountVectorizer) on the preprocessed text
# For instance, using TF-IDF as shown in the previous example
#tfidf = TfidfVectorizer(max_features=1000)
#description_features = tfidf.fit_transform(wine_reviews_df['description']).toarray()
